In [39]:
using Plots
using SpecialFunctions
using Reactive, Interact

INFO: Precompiling module Reactive.
INFO: Precompiling module Interact.


In [40]:
function target(x)
    return @. e^(-x) * cos(4π * x)
end;

function min_max(X)
    m, M = typemax(X[1]), typemin(X[1])
    for x in X
        m, M = x < m ? x : m, x > M ? x : M
    end
    return m, M
end;

"""
Finds the minimum window size for which the variation of a 1-dimensional function (represented by) converges, that is: decreases below epsilon. If Y=f(X) is a measurement then epsilon should be the measurement error.

The window is interpreted as indices in x, so is an integer. If no window is found, -1 is returned.
"""
function tuneWindow(Y, epsilon)
    N = length(Y)
    w = Int(round(N/2))
    index_found, w_found = -1, -1
    limit = Int(round(N / 8))
    while w > limit
        index_found = sliding(Y, w, epsilon)
        if index_found != -1
            w_found = w
        end
        w = Int(round(w/2))
#         w -= 1
    end
    return index_found, w_found
end

function sliding(Y, windowsize, epsilon)
    N = length(Y)
    start = -1
    if windowsize >= N
        return start
    end
    m, M = typemax(Y[1]), typemin(Y[1])
    for i in 1:N-windowsize
        if i == 1
            m, M = min_max(Y[i:i+windowsize])
        else
            old = Y[i-1]
            new = Y[i+windowsize]
            # Checking the entire window again for min max is exponential, we only need this if m or M was the first element of the previous window
            if m == old || M == old
                m, M = min_max(Y[i:i+windowsize])
            else
                m, M = new < m ? new : m, new > M ? new : M
            end
        end
        diff = abs(m - M)
        if diff < epsilon && start == -1
            start = i # Record first position where convergence is suspected
        end
        if diff >= epsilon && start != -1
            start = -1 # Variance is increasing, reset.
        end
    end
    return start
end;

In [94]:

N = 1000
X = linspace(0, 3*π , N)
Y = target(X)
m,M = min_max(Y)
epsilon = 0.1
index, windowsize = tuneWindow(Y, epsilon)
if index != -1
    x = X[index]
    y = Y[index]
    println("First threshold found at index $index where X=$x and Y=$y with windowsize=$windowsize")
    plotly()
    plot(X, Y, label="Dampened Cosine")
    plot!([x for y in Y], X/min_max(X)[2], label="Threshold with window $windowsize")
    plot!(title="Sliding window threshold with ϵ $epsilon")
    plot!(xlabel="X", ylabel="f(x)")
else
    println("No index found!!")
end

First threshold found at index 298 where X=2.8019610153638697 and Y=-0.0482060850363432 with windowsize=250


In [90]:
m = readdlm("data.txt")
cell_count, value_count = size(m)
X = 1:value_count
Ys = [ copy(m[i,:]) for i in 1:cell_count];
# results = [(0,0) for i in 1:length(Ys)]
@manipulate for ϵ in reverse(0.01:0.005:0.1)
    results = [(0,0) for i in 1:length(Ys)]
    for (cell_id, Y_i) in enumerate(Ys)
        index, windowsize = tuneWindow(Y_i, ϵ)
        if index != -1
            results[cell_id] = (index, windowsize)
        end
    end
    plotly()
    for (i, y) in enumerate(Ys)
        label = "Cell $i"
        i ==  1 ? plot(X, y, lab=label) : plot!(X, y, lab=label)
    end
    for (i, (index, window)) in enumerate(results)
        if index != 0
            x = X[index]
            plot!([x for _ in 1:value_count], X/min_max(X)[2], label="Threshold for cell $(i) and window $window")
        end
    end
    plot!(title="Variance Threshold for ϵ $ϵ")
    plot!(xlabel="Proximity Threshold (nm)", ylabel="#blinks (%)")
end


Interact.Options{:SelectionSlider,Any}(65: "input-20" = 0.055 Any , "ϵ", 0.055, "0.055", 10, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("0.1"=>0.1,"0.095"=>0.095,"0.09"=>0.09,"0.085"=>0.085,"0.08"=>0.08,"0.075"=>0.075,"0.07"=>0.07,"0.065"=>0.065,"0.06"=>0.06,"0.055"=>0.055…), Dict{Any,Any}(Pair{Any,Any}(0.055, "0.055"),Pair{Any,Any}(0.04, "0.04"),Pair{Any,Any}(0.015, "0.015"),Pair{Any,Any}(0.09, "0.09"),Pair{Any,Any}(0.08, "0.08"),Pair{Any,Any}(0.045, "0.045"),Pair{Any,Any}(0.01, "0.01"),Pair{Any,Any}(0.07, "0.07"),Pair{Any,Any}(0.06, "0.06"),Pair{Any,Any}(0.05, "0.05")…)), Any[], Any[], true, "horizontal")

In [88]:
m = readdlm("data.txt")
cell_count, value_count = size(m)
X = 1:value_count
Ys = [[ sum(m[:,i])/cell_count for i in 1:value_count]]
results = [(0,0) for i in 1:length(Ys)]
@manipulate for ϵ in reverse(0.01:0.005:0.1)
    for (cell_id, Y_i) in enumerate(Ys)
        m,M = min_max(Y_i)
        index, windowsize = tuneWindow(Y_i, ϵ)
        if index != -1
            x = X[index]
            y = Y_i[index]
            results[cell_id] = (index, windowsize)
        end
    end
    plotly()
    for (i, y) in enumerate(Ys)
        label = "Average of 9 Cells"
        i ==  1 ? plot(X, y, lab=label) : plot!(X, y, lab=label)
    end
    for (i, (index, window)) in enumerate(results)
        label = "Threshold for cell $i"
        if index != 0
            x = X[index]
            plot!([x for _ in 1:value_count], X/min_max(X)[2], label="Threshold for Average")
        end
    end
    plot!(title="Variance Threshold for ϵ $ϵ")
    plot!(xlabel="Proximity Threshold (nm)", ylabel="#blinks (%)")
end

Interact.Options{:SelectionSlider,Any}(57: "input-18" = 0.055 Any , "ϵ", 0.055, "0.055", 10, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("0.1"=>0.1,"0.095"=>0.095,"0.09"=>0.09,"0.085"=>0.085,"0.08"=>0.08,"0.075"=>0.075,"0.07"=>0.07,"0.065"=>0.065,"0.06"=>0.06,"0.055"=>0.055…), Dict{Any,Any}(Pair{Any,Any}(0.055, "0.055"),Pair{Any,Any}(0.04, "0.04"),Pair{Any,Any}(0.015, "0.015"),Pair{Any,Any}(0.09, "0.09"),Pair{Any,Any}(0.08, "0.08"),Pair{Any,Any}(0.045, "0.045"),Pair{Any,Any}(0.01, "0.01"),Pair{Any,Any}(0.07, "0.07"),Pair{Any,Any}(0.06, "0.06"),Pair{Any,Any}(0.05, "0.05")…)), Any[], Any[], true, "horizontal")